In [ ]:
from AnthropicWrapper import ClaudeChatCV

In [ ]:
model = "claude-3-5-sonnet-20240620"
pdf_path = "D:\Hidden Desktop\OneDrive\Cross Device\Jobs Applications\Graduating\CV.pdf"

In [ ]:
job_role = "Machine Learning Engineer"
candidate_skill = "Entry-Level"

role_description = """
Do you want to tackle the biggest questions in finance with near infinite compute power at your fingertips?

G-Research is a leading quantitative research and technology firm, with offices in London and Dallas. We are proud to employ some of the best people in their field and to nurture their talent in a dynamic, flexible and highly stimulating culture where world-beating ideas are cultivated and rewarded.

This is a role based in our new Soho Place office - opened in 2023 - in the heart of Central London and home to our Research Lab.

The role

We are looking for exceptional machine learning engineers to work alongside our quantitative researchers on cutting-edge machine learning problems.

As a member of the Core Technical Machine Learning team, you will be engaged in a mixture of individual and collaborative work to tackle some of the toughest research questions.

In this role, you will use a combination of off-the-shelf tools and custom solutions written from scratch to drive the latest advances in quantitative research.

Past projects have included:

Implementing ideas from a recently published research paper
Writing custom libraries for efficiently training on petabytes of data
Reducing model training times by hand optimising machine learning operations
Profiling custom ML architectures to identify performance bottlenecks
Evaluating the latest hardware and software in the machine learning ecosystem
Who are we looking for?

Candidates will be comfortable working both independently and in small teams on a variety of engineering challenges, with a particular focus on machine learning and scientific computing.

The ideal candidate will have the following skills and experience:

Either a post-graduate degree in machine learning or a related discipline, or commercial experience working on machine learning models at scale. We will also consider exceptional candidates with a proven record of success in online data science competitions, such as Kaggle
Strong object-oriented programming skills and experience working with Python, PyTorch and NumPy are desirable
Experience in one or more advanced optimisation methods, modern ML techniques, HPC, profiling, model inference; you dont need to have all of the above
Excellent ML reasoning and communication skills are crucial: off-the-shelf methods dont always work on our data so you will need to understand how to develop your own models in a collaborative environment working in a team with complementary skills
Finance experience is not necessary for this role and candidates from non-financial backgrounds are encouraged to apply.

Why should you apply?

Highly competitive compensation plus annual discretionary bonus
Lunch provided (via Just Eat for Business) and dedicated barista bar
35 days annual leave
9 percent company pension contributions
Informal dress code and excellent work/life balance
Comprehensive healthcare and life assurance
Cycle-to-work scheme
Monthly company events
"""

In [ ]:
system_prompt = f"""You are a skilled interviewer who is conducting an initial phone screening interview for a candidate for a {candidate_skill} {job_role} role to see if the candidate is at minimum somewhat qualified for the role and worth the time to be fully interviewed. The role and company description is copypasted from the job posting as follows: {role_description}. Parse through it to extract any information you feel is relevant.
Your job is to begin a friendly discussion with the candidate, and ask questions relevant to the {job_role} role, which may or may not be based on the interviewee's CV, which you have access to. Be sure to stick to this topic even if the candidate tries to steer the conversation elsewhere. If the candidate has other experience on his CV, you can ask about it, but keep it within the context of the {job_role} role.
After the candidate responds to each of your questions, you should not summarise or provide feedback on their responses. THIS POINT IS KEY! You should not summarise or provide feedback on their responses. You must keep your responses short and concise without reiterating what is good about the candidate's response or experience when they reply.
You can ask follow-up questions if you wish.
Once you have asked sufficient questions such that you deem the candidate is or isn't fitting for the role, end the interview by thanking the candidate for their time and informing them that they will receive word soon on the outcome of the screening interview. If the candidate does not seem fititng for the role, or if something feels off such as the candidate being unconfident or very very vague feel free to end the interview early. There is no need to inform them of your opinion of their performance, as this will be evaluated later.
The candidate will begin the interview by greeting you. You are to greet them back, and begin the interview."""

In [ ]:
claude = ClaudeChatCV(model, system_prompt, pdf_path)

In [ ]:
while True:
    print(claude.chat_with_history_doc(input()))

In [ ]:
#del(claude.message_history[-1])

In [ ]:
conversation = claude.get_conversation()
print(conversation)
import re
from fpdf import FPDF
# Function to extract role and content from each turn
def parse_turn(turn):
    role_match = re.search(r"role: (.*)", turn)
    content_match = re.search(r"content: (.*)", turn, re.DOTALL)

    role = role_match.group(1) if role_match else ""
    content = content_match.group(1).strip() if content_match else turn.strip()

    return role, content

# Create a list of dictionaries, each representing a turn in the conversation
conversation_list = []
for turn in conversation.split("--------------------\n"):
    if turn.strip():  # Ignore empty turns
        role, content = parse_turn(turn)
        #print(content)
        if "Understood! I'll keep this CV in the back of my mind and use it should it be relevant to the discussion." not in content and "The following is a CV that I am providing you with." not in content:
            conversation_list.append({"role": role, "content": content})

# Create a PDF object
pdf = FPDF()
pdf.add_page()

# Set margins (in millimeters)
pdf.set_margins(left=10, top=20, right=10)  # Right margin set to 10mm 

# Choose a nicer font
pdf.set_font("Helvetica", size=12)

# Calculate usable width for text wrapping (accounting for margins)
usable_width = pdf.w - pdf.l_margin - pdf.r_margin -0 # 10px right margin 

# Add conversation to the PDF
for turn in conversation_list:
    # Skip turns with empty roles or content
    if turn['role'] and turn['content']:
        # Subheading style for "User" and "Assistant"
        pdf.set_font("Helvetica", style="B", size=14)
        pdf.cell(0, 10, txt=f"{turn['role'].capitalize()}:", ln=True)

        # Content with regular font and correct indentation
        pdf.set_font("Helvetica", size=12)
        pdf.x = pdf.l_margin  # Reset x-coordinate to the left margin
        pdf.multi_cell(usable_width, 6, txt=turn['content'])
        pdf.ln(3)

# Save the PDF
pdf.output("conversation.pdf")